In [7]:
import sys
sys.path.append('..')
import gdspy
import numpy as np
from importlib import reload
#import libraries.conformal_mapping as cm
#import libraries.general_design_functions as gdf
#import libraries.general_sample_creator as creator
#from  ..libraries import general_design_functions as gdf
#fr
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator

In [8]:
watch=lambda: gdspy.LayoutViewer(depth=0,pattern={'default': 8},background='#FFFFFF') #this opens a viewer

tl_core = 20.
tl_gap = 12.
tl_ground = 10.

resonator_core = 8
resonator_gap = 7
resonator_ground = 16

pad_offset = 1000
pad_element_offset = 1000

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 300
num_couplers = 1

# Transmission line with resonator

In [9]:
reload(gdspy)
reload(creator)
reload(elements)

layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16
}
sample = creator.Sample('4_qubits',layers_configuration)

#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=9.7e3

central_line_y = sample.chip_geometry.sample_vertical_size/2

ChichkovAB = elements.AirBridge(width=15, 
                                length=60, 
                                padsize=30, 
                                min_spacing = 30,
                                layer_configuration=sample.layer_configuration)

chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads)
sample.add(chip_edge_ground)

grid_ground = elements.GridGround(sample.chip_geometry, sample.layer_configuration, sample.objects, 
                                  width=4, period=500)
sample.add(grid_ground)

# 1. Create contact pads:

p1 = elements.Pad('p1', (pad_offset,  central_line_y), np.pi, tl_core, tl_gap, tl_ground, 
                  layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                  **elements.default_pad_geometry())
p2 = elements.Pad('p2', (sample.chip_geometry.sample_horizontal_size-pad_offset,  central_line_y), 
                  0, tl_core, tl_gap, tl_ground, 
                  layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                  **elements.default_pad_geometry())
sample.add(p1)
sample.add(p2)

# 2. Create main copler:

main_coupler = elements.CPWCoupler('TL-resonator coupler', [(coupler_start, central_line_y), (coupler_start+coupler_length, central_line_y)],
                                   [resonator_core, resonator_ground, tl_core],[resonator_gap, resonator_gap, tl_gap, tl_gap],
                                   tl_ground, sample.layer_configuration, r=100)
sample.add(main_coupler)

# 3. Create fanout to create closed enfd of resonator
fanout_for_closed_end = sample.fanout(o=main_coupler, port='port1', name='closed end resonator fanout', grouping=[2,3])

# 4. ??? =(
g1 = sample.ground(o=fanout_for_closed_end, port='up', name='cl1', grounding_width=10,
               grounding_between = [(0,1)])

# 5. Create left part of transmission line
left_TL = sample.connect_cpw(o1=p1, o2=fanout_for_closed_end , port1='port', port2='center', name='left TL', points=[])

# 6. Create closed meander of resonator
closed_end_meander = sample.connect_meander(name='closed end', o1=g1, port1='narrow',
                                            meander_length=2300,restricted_scale=200, constant_scale=900, radius=25)
# 7. Create grounding of resonator
resonator_ground = sample.ground(o=closed_end_meander, port='port2', name='resonator ground', grounding_width=10, grounding_between = [(0,2)])

# 8. Create fanout to create closed enfd of resonator
fanout_for_open_end = sample.fanout(o=main_coupler, port='port2', name='open end resonator fanout', grouping=[2,3])

# 9. ??? =(
g2 = sample.ground(o=fanout_for_open_end, port='down', name='cl2', grounding_width=10,
               grounding_between = [(2,3)])
# 10. Create closed meander of resonator
open_end_resonator = elements.CPWCoupler(name='open end resonator', points=[g2.get_terminals()['narrow'].position, (g2.get_terminals()['narrow'].position[0],
                                                                                                                    g2.get_terminals()['narrow'].position[1]-500)],
                                  w=g2.get_terminals()['narrow'].w, s=g2.get_terminals()['narrow'].s,
                                  g=g2.get_terminals()['narrow'].g, layer_configuration=sample.layer_configuration, r=25)
sample.add(open_end_resonator)
# 11. Create left part of transmission line
right_TL = sample.connect_cpw(o1=p2, o2=fanout_for_open_end, port1='port', port2='center', name='right TL', points=[])

Number of curves 9.0
        Scale eff 155.55555555555554
        x 88.44444444444444
        delta 21.0
        check length 2300.0
        


In [10]:
design = sample.draw_design()
watch()

/home/alena/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  """Entry point for launching an IPython kernel.


<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [29]:
main_coupler.get_terminals()

{'port1': Type: mc-cpw, Position: (1200.0, 2350.0), Orientation: 0.0
 w: [8, 16, 20.0], s: [7, 7, 12.0, 12.0], g: 10.0
 reversed: False,
 'port2': Type: mc-cpw, Position: (1500.0, 2350.0), Orientation: 3.142
 w: [8, 16, 20.0], s: [7, 7, 12.0, 12.0], g: 10.0
 reversed: True}

In [38]:
fanout_for_closed_end.get_terminals()

{'wide': Type: mc-cpw, Position: (1200.0, 2350.0), Orientation: 3.142
 w: [8, 16, 20.0], s: [7, 7, 12.0, 12.0], g: 10.0
 reversed: True,
 'center': Type: cpw, Position: (1145.0, 2369.0), Orientation: 0.0
 w: 20.0, s: 12.0, g: 10.0
 reversed: True,
 'up': Type: mc-cpw, Position: (1177.5, 2299.0), Orientation: 1.571
 w: [16, 8], s: [7, 7, 7], g: 10.0
 reversed: True}

In [18]:
g1.get_terminals()

{'wide': Type: mc-cpw, Position: (1977.5, 2299.0), Orientation: -1.571
 w: [16, 8], s: [7, 7, 7], g: 10.0
 reversed: True,
 'narrow': Type: mc-cpw, Position: (1966.0, 2289.0), Orientation: 1.571
 w: [8], s: [7, 7], g: 10.0
 reversed: True}

In [13]:
fanout_for_open_end.get_terminals()

{'wide': Type: mc-cpw, Position: (2300.0, 2350.0), Orientation: 6.283
 w: [8, 16, 20.0], s: [7, 7, 12.0, 12.0], g: 10.0
 reversed: False,
 'down': Type: mc-cpw, Position: (2322.5, 2299.0), Orientation: 1.571
 w: [8, 16], s: [7, 7, 7], g: 10.0
 reversed: True,
 'center': Type: cpw, Position: (2355.0, 2369.0), Orientation: 3.142
 w: 20.0, s: 12.0, g: 10.0
 reversed: True}

In [5]:
g2.get_terminals()['narrow']

Type: mc-cpw, Position: (2311.0, 2289.0), Orientation: 1.571
w: [8], s: [7, 7], g: 10.0
reversed: True